# Twitter POC

En este notebook se probaran las siguientes funcionalidades:

- Obtencion de tweets de un usuario, con los replies
- Obtencion de tweets de un seguidor del mencionado usuario, con los replies
- Analisis de los tweets

In [1]:
import sys
import json
import tweepy
from pymongo import MongoClient

## Configuracion e inicializacion del API

In [2]:
config = json.load(open('config.json', 'r'))
APP_KEY = config['app_key']
APP_SECRET = config['app_secret']
ACCESS_TOKEN = config['access_token']
ACCESS_SECRET = config['access_secret']

In [3]:
def initialize_API():
    auth = tweepy.OAuthHandler(APP_KEY, APP_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET) 
    twitter_api = tweepy.API(auth)
    return twitter_api

twitter_api = initialize_API()

## Variables globales

In [4]:
user = 'MartinVizcarraC'
num_tweets = 10 
num_total_replies = 500

## Formateo de respuestas de API

El objeto Tweet que responde el API contiene muchisima informacion, como localizacion, interaccion, entre otros. Para esta demo haremos una seleccion del contenido de la respuesta.

In [5]:
def format_response(tweet):
    formatted_tweet = {}
    formatted_tweet['created_at'] = tweet.created_at
    formatted_tweet['id'] = tweet.id_str
    formatted_tweet['user'] = tweet.user.screen_name
    formatted_tweet['user_id'] = tweet.user.id_str
    formatted_tweet['text'] = tweet.full_text
    formatted_tweet['in_reply_to_status_id'] = tweet.in_reply_to_status_id_str
    formatted_tweet['in_reply_to_user_id'] = tweet.in_reply_to_user_id_str
    return formatted_tweet

## Obtener tweets 

Twitter ofrece un endpoint especifico para obtener los tweets de un usuario. 
La version Premium del API ofrece un acceso a la completa historia de tweets.
En esta demo se establece un numero de tweets. Esto debido a que dependiendo de la popularidad del usuario, se podrian obtener miles de respuestas por cada tweet.

In [9]:
def get_tweets(twitter_api, user_id=None, screen_name=None, count=5):
    tweets = []
    for tweet in tweepy.Cursor(twitter_api.user_timeline, user_id=user_id, screen_name=screen_name,
                               tweet_mode='extended').items(count):
        formatted_tweet = format_response(tweet)
        tweets.append(formatted_tweet)
    return tweets

print('Getting tweets')
user_tweets = get_tweets(twitter_api, screen_name=user, count=num_tweets)
print(user_tweets[:3])

Getting tweets
[{'created_at': datetime.datetime(2019, 9, 6, 15, 23, 57), 'id': '1169994653657178120', 'user': 'MartinVizcarraC', 'user_id': '4825296327', 'text': 'Llegamos a la frontera entre Perú y Colombia para reafirmar nuestro compromiso de preservar la Amazonía, fuente de vida y pulmón del planeta, en la #CumbrePorLaAmazonía. Antes, en Santa Rosa de Yavarí, dialogamos con sus principales defensores: los ciudadanos de esta localidad. https://t.co/OS9kKzA8Qg', 'in_reply_to_status_id': None, 'in_reply_to_user_id': None}, {'created_at': datetime.datetime(2019, 9, 2, 0, 46, 52), 'id': '1168324374891507712', 'user': 'MartinVizcarraC', 'user_id': '4825296327', 'text': 'Hoy llega a su fin #Lima2019, evento en el que demostramos que los peruanos estamos a la altura de los grandes retos y que la adversidad no nos detiene si trabajamos juntos. Gracias a todos y cada uno de los ciudadanos que lo hicieron posible. ¡Arriba, siempre #ArribaPerú! https://t.co/tGkysTpR6G', 'in_reply_to_status_id'

## Obtener respuestas

Twitter no ofrece un endpoint para obtener las respuestas en especifico a un Tweet. Si bien se pueden obtener las menciones que se hacen a un usuario, estas pueden ser no necesariamente respuestas a un tweet, sino a otros tweets, etiquetando al usuario.
Es necesario utilizar el Search API, es decir, buscar en toda el historial de tweets existente. Utilizando algunos parametros, la busqueda se reduce eficientemente

In [13]:
def get_replies(twitter_api, tweets, user_id=None, screen_name=None, result_type='mixed', num_replies=200):
    replies = []
    if len(tweets) > 0:
        # Del conjunto de tweets obtenidos en el paso anterior, buscar el id del mas antiguo
        earliest_tweet = min(tweets, key=lambda x: x['id'])
        earliest_tweet_id = earliest_tweet['id']
        query = 'to:' + screen_name
        replies = []
        # Limitar la busqueda desde el id del tweet mas antiguo
        for tweet in tweepy.Cursor(twitter_api.search, q=query, since_id=earliest_tweet_id,
                                   result_type=result_type, tweet_mode='extended').items(num_replies):
            formatted_tweet = format_response(tweet)
            replies.append(formatted_tweet)
    return replies

user_replies = get_replies(twitter_api, user_tweets, screen_name=user, num_replies=num_total_replies)
print(user_replies[:3])

[{'created_at': datetime.datetime(2019, 9, 8, 23, 15, 46), 'id': '1170838165017505792', 'user': 'GabrielaUrteaga', 'user_id': '239945406', 'text': '@MartinVizcarraC señor presidente necesitamos ayuda en Cajamarca. El lugar mas hermoso que tenemos se esta consumiendo en llamas. No tenemos lo necesario para apagar el incencio @JulianaOxenford auxilio! https://t.co/tWLYcRIU8w', 'in_reply_to_status_id': None, 'in_reply_to_user_id': '4825296327'}, {'created_at': datetime.datetime(2019, 9, 8, 23, 1, 34), 'id': '1170834593966186496', 'user': 'deleba', 'user_id': '282665265', 'text': 'RT @Gyf96Rony: @MartinVizcarraC se va a reunir con los rojos, filo terroristas de #FrenteAmplio y #NuevoPerú y una ves más demuestra que es…', 'in_reply_to_status_id': None, 'in_reply_to_user_id': None}, {'created_at': datetime.datetime(2019, 9, 8, 22, 47, 7), 'id': '1170830955805859843', 'user': 'elaguilatonchim', 'user_id': '738035172382281729', 'text': '@MartinVizcarraC Sr Pdte usted está haciendo un mandrake 

## Unir respuestas con el tweet al que responden

Entre los datos retornados para cada tweet por el API, tambien esta el id del tweet al que responden (en caso sea una respuesta). Este se puede usar para reconstruir la historia de tweets del usuario, junto con las respuestas.

In [14]:
def match_tweets_and_replies(tweets, replies):
    matched_tweets_hash = {}
    # Crearemos un hash temporalmente para optimizar la busqueda del match
    for tweet in tweets:
        tweet['replies'] = []
        tweet_id = tweet['id']
        matched_tweets_hash[tweet_id] = tweet
    # Recorremos los replies
    for reply in replies:
        in_reply_to = reply['in_reply_to_status_id']
        try:
            tweet_replied_to = matched_tweets_hash[in_reply_to]
            tweet_replied_to['replies'].append(reply)
        except KeyError:
            pass
    return list(matched_tweets_hash.values())

print('Matching tweets and replies')
matched_tweets = match_tweets_and_replies(user_tweets, user_replies)
print(matched_tweets[:3])

Matching tweets and replies
[{'created_at': datetime.datetime(2019, 9, 6, 15, 23, 57), 'id': '1169994653657178120', 'user': 'MartinVizcarraC', 'user_id': '4825296327', 'text': 'Llegamos a la frontera entre Perú y Colombia para reafirmar nuestro compromiso de preservar la Amazonía, fuente de vida y pulmón del planeta, en la #CumbrePorLaAmazonía. Antes, en Santa Rosa de Yavarí, dialogamos con sus principales defensores: los ciudadanos de esta localidad. https://t.co/OS9kKzA8Qg', 'in_reply_to_status_id': None, 'in_reply_to_user_id': None, 'replies': [{'created_at': datetime.datetime(2019, 9, 8, 22, 34, 53), 'id': '1170827875978797056', 'user': 'JesusGonzalesV2', 'user_id': '906694656855040000', 'text': '@MartinVizcarraC Sr. Presidente y cuando preserva la ciudad de tanta delicuencia. Comprometase con eso por favor.', 'in_reply_to_status_id': '1169994653657178120', 'in_reply_to_user_id': '4825296327'}, {'created_at': datetime.datetime(2019, 9, 8, 22, 18, 20), 'id': '1170823712855269377', '

## Obtener lista de seguidores

Twitter provee un endpoint para obtener los IDs de nuestros seguidores. Estos se usaran para obtener sus tweets y luego sus respuestas

In [22]:
def get_followers_ids(twitter_api, screen_name=None, max_count=200):
    followers_id_list = []
    for follower_id in tweepy.Cursor(twitter_api.followers_ids, screen_name=screen_name, stringify_ids=True).items(max_count):
        followers_id_list.append(follower_id)
    return followers_id_list
    
followers_ids = get_followers_ids(twitter_api, screen_name=user, max_count=10)
print(followers_ids)

['1161832780793147392', '1170837230404980737', '1116848664536809473', '1170834319423574016', '1170836026698457089', '1169785389038604291', '1170835645180211201', '1170832923186081792', '835605873812701185', '1169415331682672640']


## Obtener tweets y respuestas de seguidores

In [23]:
def get_user(twitter_api, user_id=None, screen_name=None):
    resp = twitter_api.get_user(user_id=user_id, screen_name=screen_name)
    user = {
        'id_str': resp.id_str,
        'screen_name': resp.screen_name
    }
    return user

def get_followers_tweets_and_replies(followers_ids):
    num_tweets_per_user = 50
    followers_data_list = []
    for f_id in followers_ids:
        follower = get_user(twitter_api, user_id=f_id)
        print('Getting data for user ' + follower['screen_name'])
        follower_tweets = get_tweets(twitter_api, screen_name=follower['screen_name'],
                                    count=num_tweets_per_user)
        follower_tweets_replies = get_replies(twitter_api, follower_tweets, screen_name=follower['screen_name'])
        matched_tweets = match_tweets_and_replies(follower_tweets, follower_tweets_replies)
        followers_data_list.append(matched_tweets)
    return followers_data_list

followers_data = get_followers_tweets_and_replies(followers_ids)
print(followers_data[:1])

Getting data for user DanielA32199706
Getting data for user DannyJr85561536
Getting data for user vela_hamilton


TweepError: Twitter error response: status code = 401